In [1]:
from datasets import DatasetDict, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from random import randrange
import pandas as pd

torch_device = "cuda" if torch.cuda.is_available() else "cpu"
print(torch_device)

/home/charlie/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda


In [2]:
eos_token='<|endoftext|>'
pad_token='<|endoftext|>'
sep_token='<|reply|>'

# add the EOS token as PAD token to avoid warnings
tokenizer = AutoTokenizer.from_pretrained("distilgpt2", return_tensors='pt', eos_token=eos_token, pad_token=pad_token)
tokenizer.padding_side = "left" 
model = AutoModelForCausalLM.from_pretrained("../output/gpt2/final", pad_token_id=tokenizer.eos_token_id).to(torch_device)

datasets = DatasetDict().load_from_disk("../data/preprocessed/dataset.hf")
decoded = [tokenizer.decode(tensor, skip_special_tokens=True)[1:-2] for tensor in datasets['validation']['input_ids']]

def create_fake(input_texts):
    full_texts = input_texts
    labels = False
    
    if randrange(100) >= 50:
        prompt_texts = [input_text.split(sep_token)[0]+sep_token for input_text in input_texts]       
        model_inputs = tokenizer(prompt_texts, return_tensors='pt', padding=True, truncation=True, max_length=1024).to(torch_device)
        # set top_k = 50 and set top_p = 0.95 and num_return_sequences = 3
        sample_outputs = model.generate(
            **model_inputs,
            max_new_tokens=40,
            do_sample=True,
            early_stopping=True,
            top_k=50,
            top_p=0.8,
            temperature=0.95,
            #num_return_sequences=1,
        )

        generated_text = tokenizer.batch_decode(sample_outputs, skip_special_tokens=True)
        full_texts = generated_text
        labels = True

    return full_texts, labels

In [3]:
# Yield successive n-sized
# chunks from l.
def divide_chunks(decoded, batch_size):
      
    # looping till length l
    for i in range(0, len(decoded), batch_size): 
        yield decoded[i:i + batch_size]
  
# How many elements each
# list should have
batch_size = 10
  
decoded_batches = list(divide_chunks(decoded, batch_size))

In [4]:
batches_with_fakes = []
num_batches = len(decoded_batches)
for i, decoded_batch in enumerate(decoded_batches):
    batch_with_fake = create_fake(decoded_batch)
    if (i%50==0):
        print(f"Batch {i} of {num_batches}")
    batches_with_fakes.append(batch_with_fake)

/home/charlie/.local/lib/python3.10/site-packages/transformers/generation/utils.py:1259: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


Batch 0 of 1388
Batch 50 of 1388
Batch 100 of 1388
Batch 150 of 1388
Batch 200 of 1388
Batch 250 of 1388
Batch 300 of 1388
Batch 350 of 1388
Batch 400 of 1388
Batch 450 of 1388
Batch 500 of 1388
Batch 550 of 1388
Batch 600 of 1388
Batch 650 of 1388
Batch 700 of 1388
Batch 750 of 1388
Batch 800 of 1388
Batch 850 of 1388
Batch 900 of 1388
Batch 950 of 1388
Batch 1000 of 1388
Batch 1050 of 1388
Batch 1100 of 1388
Batch 1150 of 1388
Batch 1200 of 1388
Batch 1250 of 1388
Batch 1300 of 1388
Batch 1350 of 1388


In [5]:
dataset_fakes_dict = {
    'text': [],
    'label': [],
}

for batch in batches_with_fakes:
    texts, label = batch[0], batch[1]
    for text in texts:
        dataset_fakes_dict['text'].append(text)
        dataset_fakes_dict['label'].append(label)

In [6]:
df = pd.DataFrame(data=dataset_fakes_dict)
df.to_csv('../data/preprocessed/fakes_validation.csv')